In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
%matplotlib inline
from tqdm import tqdm
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from sklearn import metrics
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!unzip -uq "/content/drive/My Drive/test/Copy of shopee-product-detection-dataset.zip" -d "/home/dataset"

In [ ]:
train_df=pd.read_csv('/home/dataset/train.csv')
#train_df=pd.read_csv('train.csv')
train_df.sort_values(by=['category','filename'],inplace=True)
train_df.reset_index(drop=True,inplace=True)


In [ ]:
train_df['category']=train_df['category'].astype(str)
for i in tqdm(range(train_df.shape[0])):
    full_path='/home/dataset/train/train/'+str(train_df['category'][i]).zfill(2)+'/'+train_df['filename'][i]
    #full_path='D:/train/'+str(train_df['category'][i]).zfill(2)+'/'+train_df['filename'][i]
    train_df['filename'][i]=full_path
    train_df['category'][i]=train_df['category'][i].zfill(2)


100%|██████████| 105392/105392 [02:25<00:00, 725.16it/s]


In [ ]:
train_set=train_df.sample(frac=0.8,random_state=16)
train_df=pd.concat([train_df,train_set])
test_set=train_df.drop_duplicates(keep=False)
test_set=test_set.sample(frac=1,random_state=2)
print(test_set)
print(train_set)

                                                filename category
16913  /home/dataset/train/train/06/4bdc7701f23502108...       06
30045  /home/dataset/train/train/11/500a346cec8f34b0d...       11
51405  /home/dataset/train/train/20/26d970ecb7501fc7c...       20
70873  /home/dataset/train/train/27/809957ad47990977d...       27
5010   /home/dataset/train/train/01/dbc4962162d8d459e...       01
...                                                  ...      ...
5250   /home/dataset/train/train/01/f20ed6f91e7dc8c25...       01
94485  /home/dataset/train/train/37/dedd8131094bc3570...       37
59383  /home/dataset/train/train/23/2f79c4a063721a5c5...       23
33358  /home/dataset/train/train/12/bd14d2bd85ee21258...       12
12689  /home/dataset/train/train/04/b36741d125109e0b9...       04

[21078 rows x 2 columns]
                                                 filename category
104484  /home/dataset/train/train/41/ac1b24d42f81d9f5d...       41
2655    /home/dataset/train/train/00/fd9800ac36b

In [ ]:
test_set.to_csv('/content/drive/My Drive/test_set.csv') 
train_set.to_csv('/content/drive/My Drive/train_set.csv') 

In [ ]:
img_gen=image.ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True,validation_split=0.1)
img_train = img_gen.flow_from_dataframe(
    train_set,
    shuffle=True,
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256, 256),
    batch_size=32,
    subset='training'
)

img_val = img_gen.flow_from_dataframe(
    train_set,
    shuffle=False,
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256, 256),
    batch_size=32,
    subset='validation'
)
test_datagen = image.ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
img_test = test_datagen.flow_from_dataframe(
    test_set,
    shuffle=False,
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256, 256),
    batch_size=32,
)

Found 75883 validated image filenames belonging to 42 classes.
Found 8431 validated image filenames belonging to 42 classes.
Found 21078 validated image filenames belonging to 42 classes.


In [ ]:
base_model = tf.keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(256, 256, 3))
x=inputs


# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)  # Regularize with dropout
x = layers.Dense(4096, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(42,activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.summary()


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
xception (Model)             (None, 8, 8, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 4096)              8392704   
_________________________________________________________________
dropout_16 (Dropout)         (None, 4096)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 4096)              1678

In [ ]:
opt=optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc'])
history = model.fit(
    img_train,
    epochs=20,
    validation_data=img_val
)

Epoch 1/20
2372/2372 [==============================] - 1277s 538ms/step - loss: 1.7249 - acc: 0.5356 - val_loss: 1.3495 - val_acc: 0.6223
Epoch 2/20
2372/2372 [==============================] - 1282s 540ms/step - loss: 1.4054 - acc: 0.6131 - val_loss: 1.2947 - val_acc: 0.6348
Epoch 3/20
 416/2372 [====>.........................] - ETA: 15:29 - loss: 1.3138 - acc: 0.6300

In [ ]:
model.evaluate(img_test,verbose=1)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:
predictions = model.predict(img_test,verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = img_test.classes
plt.figure(figsize = (14,10))
sns.heatmap(pd.DataFrame(metrics.confusion_matrix(true_classes, predicted_classes)), annot=True, fmt='d', cmap='YlGnBu', vmin=0)
